# 📊 WAAM Process Data: Clustering by Z using DBSCAN

This notebook processes WAAM process logs, filters active points (ArcOn), clusters the data by height (Z-axis) using the DBSCAN algorithm, and visualizes the resulting deposition layers in 3D.

## 📌 Workflow Steps:
1. Load and clean process data
2. Filter ArcOn segments only
3. Select relevant parameters
4. Cluster by Z using DBSCAN
5. Filter noise points
6. Visualize the clustered layers


Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import plotly.express as px


Load and clean data

In [2]:
import os
import pandas as pd

# 📌 Universal BASE_DIR definition
try:
    # If running as a .py file
    BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # If running in Jupyter or IPython
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))


In [3]:
# # =========================
# # 1. LOADING WORLD COORDINATES
# # =========================

# 📄 Path to points in the world coordinate system
log_path = os.path.join(BASE_DIR, 'data', 'process', 'Log_Arc_at_15-Apr-2025_17-35.csv')
df = pd.read_csv(log_path)

# 🔍 Check
# print(f"✅ Points loaded: {len(points_df)}")

# Remove extra spaces from column names
df.columns = df.columns.str.strip()


Filter active segments (ArcOn)

In [4]:
# Keep only rows where ArcOn == 1 (active deposition)
df = df[df['ArcOn'] == 1].copy()


Select relevant columns

In [5]:
columns_to_keep = ['time', 'x', 'y', 'z', 'speed', 'WFS', 'I', 'U']
df = df[columns_to_keep]


DBSCAN clustering by Z

In [6]:
# Apply DBSCAN clustering based on Z coordinate
z_values = df[['z']].values
db = DBSCAN(eps=0.5, min_samples=5).fit(z_values)

# Assign layer IDs
df['layer_id'] = db.labels_


Remove noise points

In [ ]:
# Remove noise points (label -1)
df_clean = df[df['layer_id'] != -1].copy()

# ✅ Shift all labels by +1 (to start from 1 instead of 0)
df_clean['layer_id'] = df_clean['layer_id'] + 1

# Convert layer_id to string for color grouping
df_clean['layer_id'] = df_clean['layer_id'].astype(int)


3D Visualization

In [8]:
# Visualize clustered layers in 3D
fig = px.line_3d(
    df_clean,
    x='x', y='y', z='z',
    color='layer_id',
    title="WAAM Process Layers (Clustered by Z via DBSCAN)",
    labels={'layer_id': 'Layer'},
    hover_data=['time', 'speed', 'WFS', 'I', 'U']
)
fig.update_traces(line=dict(width=3))
# fig.show()
pass


In [9]:
import os

# 📁 Path to the output directory next to this notebook
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

# 📄 Path to the HTML file
output_path = os.path.join(output_dir, "ProcessData_ClusterByZ_DBSCAN.html")

# 💾 Save the figure
fig.write_html(output_path)
print(f"✅ Plot saved: {output_path}")


✅ Plot saved: ./output\ProcessData_ClusterByZ_DBSCAN.html


Save clustered result to CSV

In [10]:
import os

output_dir = r"C:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\src\ProcessData_ClusterByZ_DBSCAN\output"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "ProcessData_ClusterByZ_DBSCAN.csv")
df_clean.to_csv(output_path, index=False)

print(f"✅ Saved: {output_path}")


✅ Saved: C:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\src\ProcessData_ClusterByZ_DBSCAN\output\ProcessData_ClusterByZ_DBSCAN.csv
